In [1]:
import torch
import shutil       
import tempfile     
from pathlib import Path
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
)
from peft import PeftModel, PeftConfig


/data/conda_envs/project1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/conda_envs/project1/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
base_model_name = "skt/kobert-base-v1"

config = AutoConfig.from_pretrained(base_model_name, num_labels=2, problem_type="single_label_classification")
base_model = AutoModelForSequenceClassification.from_pretrained(base_model_name, config=config)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
checkpoint_path = "model-checkpoints/kobert/checkpoint-1100"

model = PeftModel.from_pretrained(base_model, checkpoint_path).merge_and_unload()

In [4]:
save_dir='bnb-4bit'

In [5]:
with tempfile.TemporaryDirectory() as tmpdir:
    tmp_path = Path(tmpdir)
    model.save_pretrained(tmp_path)           # fp16 / fp32 저장
    model = AutoModelForSequenceClassification.from_pretrained(
        tmp_path, load_in_4bit=True, device_map="auto")
    
    

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [6]:
AutoTokenizer.from_pretrained(base_model_name).save_pretrained(save_dir)
model.save_pretrained(save_dir)